# Object Tracking

## Task 3: Classical Object Tracking using OpenCV

by Dechathon Niamsa-ard [st126235]

**Goal:** Implement traditional tracking algorithms using OpenCV.

**Suggested Trackers:**

- KCF
- CSRT
- MOSSE

**Instructions:**

1. Use a **short video** (e.g., pedestrians, cars, or sports).
- You can download public videos from [Pexels Videos] or [MOT Challenge dataset].
2. Define an initial bounding box.
3. Apply **two different OpenCV trackers** (e.g., KCF and CSRT).
4. Compare:
- Tracking stability
- Frame rate (FPS)
- Failure cases (drift, loss, occlusion)

**Deliverables:**

- Output video or GIF with tracking visualization
- Comparison table (FPS, success rate, drift cases)
- Short discussion on performance trade-offs

---


## Import necessary libraries

In [ ]:
# Import necessary libraries
import os
import cv2
import numpy as np
import sys
import time 

## Setup config

In [ ]:
# Set up OUTPUT_DIR
OUTPUT_DIR = './files_task3/'
test_video_path = './files_task3/test_video.mp4'

In [ ]:
# Create OUTPUT_DIR if it doesn't exist
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [ ]:
# Set up name of the output video file for each type of detection

# KCF Tracker output video file
kcf_output_video_path = os.path.join(OUTPUT_DIR, 'kcf_output_video.mp4')
# CSRT Tracker output video file
csrt_output_video_path = os.path.join(OUTPUT_DIR, 'csrt_output_video.mp4')
# MOSSE Tracker output video file
mosse_output_video_path = os.path.join(OUTPUT_DIR, 'mosse_output_video.mp4')

## Download short video

In [ ]:
# Load the video
video = cv2.VideoCapture(test_video_path)
if not video.isOpened():
    print("Error: Could not open video.")
    sys.exit()

## Define initial bounding box

In [ ]:
# Define initial bounding box


## Apply different OpenCV trackers (KCF, CSRT, MOSSE)

## Make Comparison
- Tracking stability
- Frame rate (FPS)
- Failure cases (drift, loss, occlusion)

## Short discussion on performance trade-offs